In [ ]:
import cv2
import numpy as np

# Constants
FRAME_WIDTH = 640
FRAME_HEIGHT = 480
MAX_NUM_OBJECTS = 5
MIN_OBJECT_AREA = 100
MAX_OBJECT_AREA = FRAME_HEIGHT * FRAME_WIDTH / 1.5

# Trackbar names and initial values
trackbar_values = {
    'H_MIN': 20,
    'H_MAX': 110,
    'S_MIN': 115,
    'S_MAX': 255,
    'V_MIN': 65,
    'V_MAX': 255
}
trackbar_window_name = "Trackbars"

def nothing(x):
    pass

def create_trackbars():
    cv2.namedWindow(trackbar_window_name)
    for trackbar, initial_value in trackbar_values.items():
        cv2.createTrackbar(trackbar, trackbar_window_name, initial_value, 256, nothing)

def draw_objects(contours, frame):
    for contour in contours:
        # Calculate moments for each contour
        # TODO: Could consider using this function
        # (x,y),r = cv2.minEnclosingCircle(contour)
        # print(f"Center:  {x},{y}")
        # print(r)
        moment = cv2.moments(contour)
        area = moment['m00']

        if MIN_OBJECT_AREA < area < MAX_OBJECT_AREA:
            # Calculate the center of the object
            x = int(moment['m10'] / area)
            y = int(moment['m01'] / area)

            # Fit an ellipse to the contour if possible
            if len(contour) >= 5:  # Need at least 5 points to fit an ellipse
                ellipse = cv2.fitEllipse(contour)
                center, axes, angle = ellipse
                radius = min(axes) / 2  # Use the minor axis length as the radius

                # Draw a circle around the detected object
                cv2.circle(frame, (int(center[0]), int(center[1])), int(radius), (0, 255, 0), 2)

                # Draw crosshairs
                cv2.line(frame, (int(center[0]), int(center[1])), (int(center[0]), max(int(center[1]) - 25, 0)), (0, 255, 0), 2)
                cv2.line(frame, (int(center[0]), int(center[1])), (int(center[0]), min(int(center[1]) + 25, FRAME_HEIGHT)), (0, 255, 0), 2)
                cv2.line(frame, (int(center[0]), int(center[1])), (max(int(center[0]) - 25, 0), int(center[1])), (0, 255, 0), 2)
                cv2.line(frame, (int(center[0]), int(center[1])), (min(int(center[0]) + 25, FRAME_WIDTH), int(center[1])), (0, 255, 0), 2)
                
                # Display coordinates and Radius
                cv2.putText(frame, f"{int(center[0])},{int(center[1])},{int(radius)}", (int(center[0]), int(center[1]) + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

def morph_ops(thresh):
    erode_element = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    dilate_element = cv2.getStructuringElement(cv2.MORPH_RECT, (8, 8))
    thresh = cv2.erode(thresh, erode_element, iterations=2)
    thresh = cv2.dilate(thresh, dilate_element, iterations=2)
    return thresh


# TODO: Get the radius and center of the ball. Will be used for distance estimation
# TODO: Use get_ball_location() to estimate the location of each detected ball
def track_filtered_objects(threshold, frame):
    temp = threshold.copy()
    contours, _ = cv2.findContours(temp, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        num_objects = len(contours)
        if num_objects < MAX_NUM_OBJECTS:
            draw_objects(contours, frame)
        else:
            cv2.putText(frame, "TOO MUCH NOISE! ADJUST FILTER", (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    


# Returns straight line distance of ball from camera
def get_ball_location(camera_matrix, detected_obj):
    # camera_matrix: (3x3 numpy array) with element 0,0 being the focal length x 
    # detected_obj: (dict) Contains keys 'radius' and 'center', referring to the measured radius and pixel coordinates of the center of the ball
    
    true_radius = 0.0335 # Actual radius of tennis ball in meters 
    # TODO: Focal length in horizontal direction
    # (640/2)*tan(70/2) For raspbery pi camera
    f_x = 224.0664122

    # Distance of ball based om measured radius 
    # distance = true_radius / predicted_radius * focal_length
    return true_radius/detected_obj['radius'] * f_x


def main():
    create_trackbars()
    capture = cv2.VideoCapture(0)

    if not capture.isOpened():
        print("Error: Could not open video capture")
        return

    capture.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
    capture.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

    while True:
        ret, frame = capture.read()
        if not ret:
            print("Error: Empty frame captured")
            break

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        hsv_values = {key: cv2.getTrackbarPos(key, trackbar_window_name) for key in trackbar_values}
        threshold = cv2.inRange(hsv, (hsv_values['H_MIN'], hsv_values['S_MIN'], hsv_values['V_MIN']),
                                         (hsv_values['H_MAX'], hsv_values['S_MAX'], hsv_values['V_MAX']))

        threshold = morph_ops(threshold)

        track_filtered_objects(threshold, frame)

        cv2.imshow("Thresholded Image", threshold)
        cv2.imshow("Original Image", frame)
        cv2.imshow("HSV Image", hsv)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    capture.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)


if __name__ == "__main__":
    main()


In [ ]:
# Testing yellow filter
yellow_lower = np.array([20, 100, 100])
yellow_upper = np.array([30, 255, 255])
img_threshed_yellow = cv2.inRange(hsv, yellow_lower, yellow_upper) # Returns pixels that are within the yellow range

In [ ]:
import cv2 
import numpy as np

# Constants
FRAME_WIDTH = 640
FRAME_HEIGHT = 480
MAX_NUM_OBJECTS = 5
MIN_OBJECT_AREA = 100
MAX_OBJECT_AREA = FRAME_HEIGHT * FRAME_WIDTH / 1.5

# Trackbar names and initial values
trackbar_values = {
    'H_MIN': 20,
    'H_MAX': 110,
    'S_MIN': 115,
    'S_MAX': 255,
    'V_MIN': 65,
    'V_MAX': 255
}
trackbar_window_name = "Trackbars"

capture = cv2.VideoCapture(0)

if not capture.isOpened():
    print("Error: Could not open video capture")

capture.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)
# capture.set(cv2.CAP_PROP_EXPOSURE, 30)


# HSV values
# yellow_lower = np.array([20, 100, 100])
# yellow_upper = np.array([30, 255, 255])

# Good values for tennis balls. Further testing fo balls far away
yellow_lower = (28, 50, 100)
yellow_upper = (55, 255, 255)


white_lower = np.array([0,100,180])
white_upper = np.array([255,255,255])

# HLS Values
# white_lower = np.uint8([0, 200, 0])
# white_upper = np.uint8([255, 255, 255])

# yellow_lower = np.uint8([10, 0,   100])
# yellow_upper = np.uint8([30, 255, 180])

while True:
    ret, frame = capture.read()
    if not ret:
        print("Error: Empty frame captured")
        break

    # Yellow filter
    # hls = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS)
    # mask = cv2.inRange(hls, yellow_lower, yellow_upper)
    
    # Thresholding in HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, white_lower, white_upper)

    # Threholding in HLS
    # hls = cv2.cvtColor(frame, cv2.COLOR_BGR2HLS)
    # lower = np.uint8([0, 200, 0])
    # upper = np.uint8([255, 255, 255])

    # #change 250 to lower numbers to include more values as "white"
    # mask = cv2.inRange(hls, lower, upper)
                                                                                                                                                                                                                                        
    # mask = cv2.inRange(frame, (230, 230, 230), (255, 255, 255))

    cv2.imshow("Original", frame)
    cv2.imshow("HSV", hsv)
    cv2.imshow("Threshold",mask)
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        capture.release()
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        break



In [127]:
import cv2
import numpy as np

# Known parameters
r_true = 0.0035  # The radius of the tennis ball (m)
f = 100

# HSV values for ball
lower_yellow_green = np.array([28, 50, 100])
upper_yellow_green = np.array([55, 255, 255])

MIN_BALL_RADIUS = 10
MAX_NUM_OBJECTS = 5
MIN_BALL_AREA = 200
MAX_OBJECT_AREA = 2000



def detect_ball(frame):
    # Convert the image to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Get mask with HSV values
    mask = cv2.inRange(hsv, lower_yellow_green, upper_yellow_green)
    # Morph ops
    # mask = cv2.erode(mask, None, iterations=3)
    # mask = cv2.dilate(mask, None, iterations=3)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8,8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=1)
    cv2.imshow("Mask", mask)
    cv2.waitKey(0)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Extract frame size and get the center coordinates of the frame tuple: (cols,rows)
    frame_center = tuple(x//2 for x in frame.shape[:2])[::-1]
    cv2.circle(frame, (frame_center), 20, (255,0,0),-1)

    if len(contours): print(len(contours))  # Number of contours
    if contours and len(contours)<MAX_NUM_OBJECTS:
    #     # Handle multple balls in frame
        for i,contour in enumerate(contours):
            # Calculate moments for each contour
            moment = cv2.moments(contour)
            area = moment['m00']
            print(f"Area of contour {i} = {area}")
            
            if MIN_BALL_AREA < area:
                # Center of the object
                # x = int(moment['m10'] / area)
                # y = int(moment['m01'] / area)

                # Using ellipse 
                if len(contour) >= 5:  # At least 5 points to fit an ellipse
                    ellipse = cv2.fitEllipse(contour)
                    center, axes, angle = ellipse
                    center = np.array(center).astype("int")
                    radius = min(axes) / 2 # Use the major axis length as the radius
                    cv2.circle(frame, center, int(radius), (255, 0, 0), 2)
                    cv2.circle(frame, center, 5, (0, 0, 255), -1)

                # Using circle
                (x, y), radius = cv2.minEnclosingCircle(contour)
                center = (int(x), int(y))
                print(f"R{i} = {radius}")
                cv2.circle(frame, center, int(radius), (0, 255, 0), 2)
                cv2.circle(frame, center, 5, (0, 0, 255), -1)
                cv2.imshow("Contours", frame)
                cv2.waitKey(0)
                    
                    
            # Get the minimum enclosing circle for the largest contour
            # (x, y), radius = cv2.minEnclosingCircle(contour)
            # center = (int(x), int(y))
            # radius = int(radius)
            # # Calculate distance from the ball
            # if radius > MIN_BALL_RADIUS:
            #     distance = (r_true * f) / radius
            #     cv2.circle(frame, center, radius, (0, 255, 0), 2)

    
    # if contours and len(contours)<MAX_NUM_OBJECTS:
    #     # Single ball 
    #     # Find the largest contour which will be the tennis ball
    #     largest_contour = max(contours, key=cv2.contourArea)
        
    #     # Get the minimum enclosing circle for the largest contour
    #     (x, y), radius = cv2.minEnclosingCircle(largest_contour)
    #     center = (int(x), int(y))
    #     radius = int(radius)

    #     # Draw circle and center of ball
    #     cv2.circle(frame, center, radius, (0, 255, 0), 2)
    #     cv2.circle(frame, center, 5, (0, 0, 255), -1)

    #     # Calculate distance from the ball
    #     if radius > MIN_BALL_RADIUS:
    #         distance = (r_true * f) / radius
    #         # Calculate the x, y coordinates relative to the center of the frame
    #         # x_coord = center[0] - frame_center_x
    #         x_offset = center[0] - frame_center[0]
    #         y_offset = np.sqrt((distance**2) - (x_offset**2))
    #         # y_coord = center[1] - frame_center_y
    #         # offset = np.subtract(center, frame_center)

    #         cv2.putText(frame, 
    #                     f"{center}, {radius}", 
    #                     (int(center[0]), int(center[1]) + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    #         return distance, x_offset, frame
    #     else:
    #         distance = None

    return None, None, frame

# Testing video
# Capture video from camera (or load a test image for simplicity)
# cap = cv2.VideoCapture(0)

# Testing image
img = cv2.imread('ball3.jpg')

cv2.imshow("Image", img)
cv2.waitKey(0)
detect_ball(img)

# cap = cv2.VideoCapture(0)
# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
    
#     distance, coordinates, frame = detect_ball(frame)

#     # Display the resulting frame
#     cv2.imshow('Tennis Ball', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# Release the capture and destroy all windows
# cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

2
Area of contour 0 = 109750.5
R0 = 194.61843872070312
Area of contour 1 = 126.0


-1

In [44]:
# Line detection
# Python program to illustrate HoughLine
# method for line detection
import cv2
import numpy as np

# img = cv2.imread('test1.jpg')

# cv2.imshow("Image", img)
# cv2.waitKey(0)

# RGB values
# lower_white = (180,180,180)
# upper_white = (255,255,255)

# HSV values
sensitivity = 30
lower_white = np.array([0, 0, 200])
upper_white = np.array([180, 50, 255])

# Testing with video
cap = cv2.VideoCapture('test.mp4')
while True:
    ret, img = cap.read()

    if not ret:
        break
    (H, W,C) = img.shape
    img = img[150:650, :, :]
    cv2.imshow("Image", img)

# Convert the img to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    eq = cv2.equalizeHist(gray) # Histogram equalization
    cv2.imshow("Equalization", eq)


    # smooth = cv2.medianBlur(img, 3)
    mask = cv2.inRange(eq, 235, 255)
    # mask = cv2.inRange(hsv, lower_white, upper_white)
    # mask = cv2.inRange(img, lower_white, higher_white)
    # print(mask.shape)
    # gray = img
    cv2.imshow("mask", mask)
    cv2.waitKey(0)

    # Apply edge detection method on the image
    edges = cv2.Canny(mask, 200, 255, apertureSize=5)
    cv2.imshow("edges", edges)
    cv2.waitKey(0)
    lines_list = []

    # This returns an array of r and theta values
    lines = cv2.HoughLinesP(mask, # Input edge image
                          1, # Distance resolution in pixels
                          np.pi/360, # Angle resolution in radians (1 degree)
                          threshold=150, # Min number of votes for valid line
                          minLineLength=200, # Min allowed length of line
                          maxLineGap=15 # Max allowed gap between line for joining them
    )

    if lines is not None:
      # print(lines.shape)
      # cv2.destroyAllWindows()
      # cv2.waitKey(1)
      for points in lines:
          # Extracted points nested in the list
        x1,y1,x2,y2=points[0]
        # Draw the lines joing the points
        # On the original image
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
        # Maintain a simples lookup list for points
        lines_list.append([(x1,y1),(x2,y2)])

    cv2.imshow("detected lines", img)
    # cv2.imwrite('linesDetected.jpg', img)

#The below for loop runs till r and theta values
# are in the range of the 2d array
# for r_theta in lines:
#     arr = np.array(r_theta[0], dtype=np.float64)
#     r, theta = arr
#     # Stores the value of cos(theta) in a
#     a = np.cos(theta)

#     # Stores the value of sin(theta) in b
#     b = np.sin(theta)

#     # x0 stores the value rcos(theta)
#     x0 = a*r

#     # y0 stores the value rsin(theta)
#     y0 = b*r

#     # x1 stores the rounded off value of (rcos(theta)-1000sin(theta))
#     x1 = int(x0 + 10000*(-b))

#     # y1 stores the rounded off value of (rsin(theta)+1000cos(theta))
#     y1 = int(y0 + 10000*(a))

#     # x2 stores the rounded off value of (rcos(theta)+1000sin(theta))
#     x2 = int(x0 - 10000*(-b))

#     # y2 stores the rounded off value of (rsin(theta)-1000cos(theta))
#     y2 = int(y0 - 10000*(a))

#     # cv2.line draws a line in img from the point(x1,y1) to (x2,y2).
#     # (0,0,255) denotes the colour of the line to be
#     # drawn. In this case, it is red.
#     cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

# All the changes made in the input image are finally
# written on a new image houghlines.jpg

# cv2.imshow("detected lines", img)
# cv2.imwrite('linesDetected.jpg', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [45]:
cv2.destroyAllWindows()
cv2.waitKey(1)

-1